In [1]:
import os, argparse, json, re
from collections import defaultdict
import customutils
import pandas as pd
import numpy as np

LABELS = customutils.get_classes('configs/halpe26_classes.txt')
LABELS_COCOWB = customutils.get_classes('configs/cocowb_classes.txt')
LABELS_COCO = customutils.get_classes('configs/coco_classes.txt')
LABELS_FOOT = ['LBigToe','RBigToe','LSmallToe','RSmallToe','LHeel','RHeel']
LABELS_MISSING = ['Head','Neck','Hip']

df_running_annotations = customutils.load_images_dataframe()
df_running_annotations

,image_id,pose_id,file_name,frame,person
0,1000,1000,Athletics_Mixed_Tokyo_2020_20_1.mp4,0,0
1,1001,1001,Athletics_Mixed_Tokyo_2020_20_1.mp4,1,0
2,1002,1002,Athletics_Mixed_Tokyo_2020_20_1.mp4,2,0
3,1003,1003,Athletics_Mixed_Tokyo_2020_20_1.mp4,3,0
4,1004,1004,Athletics_Mixed_Tokyo_2020_20_1.mp4,4,0
...,...,...,...,...,...
12105,45160,145160,World_Athletics_Women_Marathon_Oregon_2022_8.mp4,160,1
12106,45161,145161,World_Athletics_Women_Marathon_Oregon_2022_8.mp4,161,1
12107,45162,145162,World_Athletics_Women_Marathon_Oregon_2022_8.mp4,162,1
12108,45163,145163,World_Athletics_Women_Marathon_Oregon_2022_8.mp4,163,1


In [2]:
df_running_annotations[df_running_annotations['image_id'] == 46085]

,image_id,pose_id,file_name,frame,person


In [3]:
df = customutils.load_keypoints_dataframe()
df[df.duplicated(subset=['id'],keep=False)]

,id,image_id,category_id,bbox,area,iscrowd,num_keypoints,keypoints


In [4]:

df = df.drop(columns=['iscrowd', 'num_keypoints'])
df = df.set_index(['id'])
df

,image_id,category_id,bbox,area,keypoints
id,,,,,
1000,1000,1,"[957.5031, 204.6478, 294.4987, 641.9175]",189043.869257,"[1132.7201, 271.7583, 1, 1151.8834, 256.5806, ..."
1001,1001,1,"[895.2875, 201.4572, 396.59609999999986, 639.6...",253699.828317,"[1135.4403, 262.4473, 1, 1151.8834, 253.39, 1,..."
1002,1002,1,"[876.5417, 207.8383, 455.8919000000001, 641.2885]",292358.232713,"[1132.7201, 255.8056, 1, 1148.6928, 247.0089, ..."
1003,1003,1,"[892.6316, 187.0998, 445.91919999999993, 658.8...",293787.800419,"[1129.5295, 249.4245, 1, 1145.5023, 240.6278, ..."
1004,1004,1,"[912.8355, 201.4572, 406.572, 681.1703]",276944.771212,"[1132.7201, 249.4245, 1, 1148.6928, 240.6278, ..."
...,...,...,...,...,...
145160,45160,1,"[749.0932, 271.5931, 241.01109999999994, 816.5...",196795.299998,"[811.8716, 319.3694, 1, 827.6156, 303.8024, 1,..."
145161,45161,1,"[749.0932, 271.5931, 241.01109999999994, 816.5...",196795.299998,"[811.8716, 319.3694, 1, 827.6156, 303.8024, 1,..."
145162,45162,1,"[749.0932, 271.5931, 241.01109999999994, 816.5...",196795.299998,"[811.8716, 319.3694, 1, 827.6156, 303.8024, 1,..."


In [5]:

df_categories = customutils.load_categories_dataframe()
kpts_classes = df_categories[df_categories['id'] == 1]['keypoints'][0]

In [6]:

df['dist_vitpose'] = np.ones((len(df),1)) * np.inf
df['dist_cotracker'] = np.ones((len(df),1)) * np.inf
df['dist_cotracker3'] = np.ones((len(df),1)) * np.inf
df['oks_vitpose'] = float(0.0)
df['oks_cotracker'] = float(0.0)
df['oks_cotracker3'] = float(0.0)
df['oks_coco_vitpose'] = float(0.0)
df['oks_coco_cotracker'] = float(0.0)
df['oks_coco_cotracker3'] = float(0.0)
df['oks_foot_vitpose'] = float(0.0)
df['oks_foot_cotracker'] = float(0.0)
df['oks_foot_cotracker3'] = float(0.0)
df['idx'] = ""
df

,image_id,category_id,bbox,area,keypoints,dist_vitpose,dist_cotracker,dist_cotracker3,oks_vitpose,oks_cotracker,oks_cotracker3,oks_coco_vitpose,oks_coco_cotracker,oks_coco_cotracker3,oks_foot_vitpose,oks_foot_cotracker,oks_foot_cotracker3,idx
id,,,,,,,,,,,,,,,,,,
1000,1000,1,"[957.5031, 204.6478, 294.4987, 641.9175]",189043.869257,"[1132.7201, 271.7583, 1, 1151.8834, 256.5806, ...",inf,inf,inf,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,
1001,1001,1,"[895.2875, 201.4572, 396.59609999999986, 639.6...",253699.828317,"[1135.4403, 262.4473, 1, 1151.8834, 253.39, 1,...",inf,inf,inf,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,
1002,1002,1,"[876.5417, 207.8383, 455.8919000000001, 641.2885]",292358.232713,"[1132.7201, 255.8056, 1, 1148.6928, 247.0089, ...",inf,inf,inf,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,
1003,1003,1,"[892.6316, 187.0998, 445.91919999999993, 658.8...",293787.800419,"[1129.5295, 249.4245, 1, 1145.5023, 240.6278, ...",inf,inf,inf,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,
1004,1004,1,"[912.8355, 201.4572, 406.572, 681.1703]",276944.771212,"[1132.7201, 249.4245, 1, 1148.6928, 240.6278, ...",inf,inf,inf,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145160,45160,1,"[749.0932, 271.5931, 241.01109999999994, 816.5...",196795.299998,"[811.8716, 319.3694, 1, 827.6156, 303.8024, 1,...",inf,inf,inf,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,
145161,45161,1,"[749.0932, 271.5931, 241.01109999999994, 816.5...",196795.299998,"[811.8716, 319.3694, 1, 827.6156, 303.8024, 1,...",inf,inf,inf,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,
145162,45162,1,"[749.0932, 271.5931, 241.01109999999994, 816.5...",196795.299998,"[811.8716, 319.3694, 1, 827.6156, 303.8024, 1,...",inf,inf,inf,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,


In [7]:
def edit_keypoints(kpts):
    kpts = np.array(kpts).reshape(-1,3).astype(float)
    vi = kpts[:,2]
    kpts = kpts[:,0:2]
    return kpts, vi

def get_keypoint(kpts, id):
    index = kpts_classes.index(id)
    return [kpts[index*3],kpts[(index*3)+1]]

def load_keypoints(root, model):
    dist = 'dist_' + str.lower(model)
    conf = 'conf_' + str.lower(model)
    oks = 'oks_' + str.lower(model)
    oks_coco = 'oks_coco_' + str.lower(model)
    oks_foot = 'oks_foot_' + str.lower(model)
    file = os.path.join(root, model, 'person_keypoints_running.json')
    with open(file, 'r') as f:
        predictions = json.load(f)
        d = {}
        
        empty_keypoint = [0, 0, 0]        

        for prediction in predictions:
            # if prediction['image_id'] != 46162:
            #     continue
            pose_id = -1
            mask_frame = df['image_id'] == prediction['image_id']
            gts = df[mask_frame]
            dt = prediction['keypoints']
            # check if we have missing joints (compared to GT)
            if len(dt) < len(LABELS)*3:
                i = 0
                for label in LABELS:
                    if label not in LABELS_COCOWB:
                        dt[i:i] = empty_keypoint
                    i += 3

            kpts1, vi1 = edit_keypoints(dt)
            min_dist = np.inf #np.ones((len(kpts1),1)) * 
            
            # for i in range(len(gts)):
            for pose_id in gts.index:
                gt = gts['keypoints'][pose_id]
                kpts2, vi2 = edit_keypoints(gt)
                d = np.linalg.norm(kpts1 - kpts2, ord=2, axis=1)
                v = np.ones(len(d))

                for part in range(len(d)):
                    if vi1[part] == 0 or vi2[part] == 0:
                        d[part] = 0
                        v[part] = 0
                                        
                area = customutils.compute_area_keypoints(gt)

                # compute head size for distance normalization
                head = get_keypoint(gt,"head")
                neck = get_keypoint(gt,"neck")

                headSize = 1
                if (len(head) > 0 and len(neck) > 0):
                    headSize = customutils.get_head_size(head[0], head[1], neck[0], neck[1])
                # normalize distance
                dNorm = np.sum(d)/headSize

                oks_val = customutils.OKS(gt, dt, area, 0, 26)
                oks_val_coco = customutils.OKS(gt, dt, area, 0, 17)
                oks_val_foot = customutils.OKS(gt, dt, area, 20, 26)

                if dNorm < min_dist and dNorm < gts[dist][pose_id]:
                    min_dist = dNorm
                    df.loc[pose_id, dist] = min_dist
                    df.loc[pose_id, oks] = oks_val
                    df.loc[pose_id, oks_coco] = oks_val_coco
                    df.loc[pose_id, oks_foot] = oks_val_foot
                    if 'score' in prediction:
                        df.loc[pose_id, conf] = prediction['score']
                    else:
                        df.loc[pose_id, conf] = 1
                    if 'idx' in prediction:
                        df.loc[pose_id, 'idx'] = prediction['idx']
            
        # df_running_annotations['score_' + str.lower(model)] = df_running_annotations['image_id'].map(d)

In [8]:
root = '../data/videos/results/'
load_keypoints(root, 'cotracker')
load_keypoints(root, 'cotracker3')

In [9]:
root = '../videos/results/'
load_keypoints(root, 'vitpose')

In [10]:
# remove unmatched poses/frames
df = df.loc[(df['conf_vitpose'] > float(0))]
df

,image_id,category_id,bbox,area,keypoints,dist_vitpose,dist_cotracker,dist_cotracker3,oks_vitpose,oks_cotracker,...,oks_coco_vitpose,oks_coco_cotracker,oks_coco_cotracker3,oks_foot_vitpose,oks_foot_cotracker,oks_foot_cotracker3,idx,conf_cotracker,conf_cotracker3,conf_vitpose
id,,,,,,,,,,,,,,,,,,,,,
1000,1000,1,"[957.5031, 204.6478, 294.4987, 641.9175]",189043.869257,"[1132.7201, 271.7583, 1, 1151.8834, 256.5806, ...",6.448374,7.334497,6.448370,0.835930,0.838527,...,0.833253,0.867140,0.833253,0.843515,0.734421,0.843515,2,1.0,1.0,0.808579
1001,1001,1,"[895.2875, 201.4572, 396.59609999999986, 639.6...",253699.828317,"[1135.4403, 262.4473, 1, 1151.8834, 253.39, 1,...",5.973131,6.483824,5.973130,0.892015,0.870079,...,0.895465,0.834184,0.895465,0.882239,0.967516,0.882239,2,1.0,1.0,0.830877
1002,1002,1,"[876.5417, 207.8383, 455.8919000000001, 641.2885]",292358.232713,"[1132.7201, 255.8056, 1, 1148.6928, 247.0089, ...",4.706753,9.820727,7.164789,0.924794,0.769824,...,0.906664,0.830779,0.893127,0.976162,0.793896,0.797956,2,1.0,1.0,0.866709
1003,1003,1,"[892.6316, 187.0998, 445.91919999999993, 658.8...",293787.800419,"[1129.5295, 249.4245, 1, 1145.5023, 240.6278, ...",3.991620,9.536559,7.421590,0.941594,0.850269,...,0.927926,0.853897,0.879016,0.980320,0.787800,0.814763,2,1.0,1.0,0.879785
1004,1004,1,"[912.8355, 201.4572, 406.572, 681.1703]",276944.771212,"[1132.7201, 249.4245, 1, 1148.6928, 240.6278, ...",5.594584,13.224239,9.600370,0.921789,0.771087,...,0.929047,0.803177,0.873194,0.901227,0.731714,0.756604,2,1.0,1.0,0.807213
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145159,45159,1,"[749.0932, 271.5931, 241.01109999999994, 816.5...",196795.299998,"[811.8716, 319.3694, 1, 827.6156, 303.8024, 1,...",6.092382,inf,6.240648,0.807955,0.000000,...,0.765260,0.000000,0.766484,0.928922,0.000000,0.909397,2,NaN,1.0,0.839506
145160,45160,1,"[749.0932, 271.5931, 241.01109999999994, 816.5...",196795.299998,"[811.8716, 319.3694, 1, 827.6156, 303.8024, 1,...",5.988541,inf,5.948414,0.813465,0.000000,...,0.780822,0.000000,0.774624,0.905955,0.000000,0.909435,2,NaN,1.0,0.837648
145161,45161,1,"[749.0932, 271.5931, 241.01109999999994, 816.5...",196795.299998,"[811.8716, 319.3694, 1, 827.6156, 303.8024, 1,...",5.262168,inf,5.775690,0.841295,0.000000,...,0.808669,0.000000,0.779914,0.933736,0.000000,0.912991,2,NaN,1.0,0.844572


In [13]:
def computeAP(oks, df, thresh):
    T = len(df[oks])
    idxs = np.argwhere(df[oks] >= thresh)
    ap = 100.0*len(idxs)/T
    return ap

def computeAllAP(oks, df):
    distThresh = 0.5
    ap50 = computeAP(oks, df, distThresh)
    ap = np.copy(ap50)
    for i in range(9):
        distThresh += 0.05
        ap += computeAP(oks, df, distThresh)
    ap75 = computeAP(oks, df, 0.75)
    ap = ap/10
    print('AP (' + oks + '): ' + str(ap))
    #print('AP0.5 (' + oks + '): ' + str(ap50))
    #print('AP0.75 (' + oks + '): ' + str(ap75))


In [14]:
computeAllAP('oks_vitpose', df)
computeAllAP('oks_coco_vitpose', df)
computeAllAP('oks_foot_vitpose', df)
computeAllAP('oks_cotracker', df)
computeAllAP('oks_coco_cotracker', df)
computeAllAP('oks_foot_cotracker', df)
computeAllAP('oks_cotracker3', df)
computeAllAP('oks_coco_cotracker3', df)
computeAllAP('oks_foot_cotracker3', df)

AP (oks_vitpose): 67.7093494254774
AP (oks_coco_vitpose): 69.93634785484005
AP (oks_foot_vitpose): 64.21592130280234
AP (oks_cotracker): 7.620897743242127
AP (oks_coco_cotracker): 9.025378192940398
AP (oks_foot_cotracker): 5.807224931801273
AP (oks_cotracker3): 33.83070182689923
AP (oks_coco_cotracker3): 37.463007357196
AP (oks_foot_cotracker3): 30.256261883111517


In [12]:
pd.set_option('display.max_rows', None)

# get frames where solution is better than original
df_good = df[(df['oks_cotracker'] > df['oks_vitpose']) | (df['oks_foot_cotracker'] > df['oks_foot_alphapose'])][['image_id','oks_alphapose', 'oks_cotracker','oks_foot_alphapose','oks_foot_cotracker']]

In [16]:
df_good[df_good['oks_foot_cotracker'] > 0.9]

,image_id,oks_alphapose,oks_cotracker,oks_foot_alphapose,oks_foot_cotracker
id,,,,,
1005,1005,0.906173,0.906173,0.935919,0.935920
1006,1006,0.587348,0.859913,0.000000,0.900384
1018,1018,0.872862,0.893110,0.962639,0.968542
1020,1020,0.774531,0.806591,0.695097,0.941477
1026,1026,0.934813,0.871543,0.964829,0.972247
1027,1027,0.761855,0.820861,0.481696,0.972919
1028,1028,0.846857,0.792953,0.636388,0.915989
101001,1001,0.913999,0.913999,0.962782,0.962783
101002,1002,0.763998,0.861731,0.481767,0.925029


In [11]:
df2 = df[['oks_vitpose', 'oks_cotracker','oks_foot_vitpose','oks_foot_cotracker']].mean()
df2

oks_alphapose         0.708926
oks_cotracker         0.451423
oks_foot_alphapose    0.647285
oks_foot_cotracker    0.379899
dtype: float64

In [ ]:
def load_results(root, model):
    file = os.path.join(root, 'results_pckh_' + model + '.json')
    with open(file, 'r') as f:
        predictions = json.load(f)
        d = {}
        for prediction in predictions:
            d[prediction['image_id']] = prediction['score']
            
        df_running_annotations['score_' + str.lower(model)] = df_running_annotations['image_id'].map(d)

In [ ]:
# remove unmatched poses/frames
df_excl_hidden = df.loc[(df['oks_foot_vitpose'] > 0.01) | (df['oks_foot_openpose'] > 0.01) | (df['oks_foot_vitpose'] > 0.01)]
df_excl_hidden

,image_id,category_id,bbox,area,keypoints,dist_alphapose,dist_openpose,dist_vitpose,conf_alphapose,conf_openpose,...,oks_vitpose,oks_coco_alphapose,oks_coco_openpose,oks_coco_vitpose,oks_foot_alphapose,oks_foot_openpose,oks_foot_vitpose,kpts_alphapose,kpts_openpose,kpts_vitpose
id,,,,,,,,,,,,,,,,,,,,,
1000,1000,1,"[957.5, 204.65, 294.5, 641.9200000000001]",189045.4400,"[1132.72, 271.76, 1, 1151.88, 256.58, 1, 1137....",7.334525,7.013630,6.367525,0.877560,0.683721,...,0.839584,0.867149,0.854623,0.837240,0.734402,0.742966,0.846226,"1136.07421875,266.266845703125,2,1155.60852050...","1139.55,268.183,2,1160.02,256.521,2,1139.39,24...","1135.575439453125,264.9145202636719,2,1153.360..."
1001,1001,1,"[895.29, 201.46, 396.59000000000015, 639.68999...",253694.6571,"[1132.72, 268.57, 1, 1151.88, 253.39, 1, 1125....",6.929502,7.620146,8.830845,0.867543,0.711724,...,0.814032,0.815075,0.832625,0.820795,0.963230,0.866944,0.794871,"1134.0335693359375,257.7659912109375,2,1153.15...","1139.57,259.547,2,1157.16,247.864,2,1139.4,244...","1133.6328125,259.5325622558594,2,1151.58813476..."
1002,1002,1,"[882.53, 207.84, 459.21000000000004, 641.29]",294486.7809,"[1132.72, 255.81, 1, 1148.69, 247.01, 1, 1132....",6.324701,5.498150,4.783507,0.869874,0.753456,...,0.922007,0.872357,0.903906,0.910690,0.978284,0.971253,0.954072,"1128.94189453125,252.69618225097656,2,1148.562...","1139.6,247.833,2,1157.25,244.735,2,1139.4,235....","1136.784423828125,251.25259399414062,2,1154.65..."
1003,1003,1,"[879.33, 187.1, 459.2199999999999, 658.84]",302552.5048,"[1129.53, 249.42, 1, 1145.5, 240.63, 1, 1129.1...",4.901953,5.589828,4.732465,0.887327,0.755095,...,0.929315,0.907709,0.905083,0.918811,0.977284,0.967039,0.959075,"1129.5250244140625,242.39736938476562,2,1143.6...","1136.53,244.883,2,1148.4,241.765,2,1133.67,227...","1132.8712158203125,246.05426025390625,2,1151.2..."
1004,1004,1,"[912.84, 201.46, 406.57000000000005, 681.17]",276943.2869,"[1132.72, 249.42, 1, 1148.69, 240.63, 1, 1132....",5.010580,5.075816,5.901758,0.891673,0.756369,...,0.919301,0.922700,0.956517,0.943024,0.968041,0.950892,0.852084,"1127.0386962890625,248.37823486328125,2,1148.4...","1133.7,244.898,2,1148.37,241.799,2,1133.6,227....","1129.1875,247.21041870117188,2,1149.0015869140..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
146159,46159,1,"[1043.37, 309.11, 277.4100000000001, 689.9]",191385.1590,"[1125.14, 361.38, 1, 1135.3, 345.97, 1, 1109.3...",7.501191,7.240335,6.082551,0.868441,0.675163,...,0.884817,0.764059,0.764995,0.891781,0.910295,0.886464,0.865086,"1141.66064453125,357.68817138671875,2,1148.506...","1118.98,374.317,2,1133.63,362.501,2,1115.9,362...","1122.9000244140625,359.6590270996094,2,1136.55..."
146160,46160,1,"[1043.37, 309.11, 277.4100000000001, 689.9]",191385.1590,"[1125.14, 361.38, 1, 1135.3, 345.97, 1, 1109.3...",7.738347,7.812317,6.460968,0.875172,0.663894,...,0.869753,0.732880,0.732826,0.870795,0.944972,0.881181,0.866799,"1143.8115234375,357.8825988769531,2,1150.61694...","1118.99,377.24,2,1133.62,362.577,2,1115.92,365...","1124.10986328125,359.1108093261719,2,1138.1390..."
146161,46161,1,"[1043.37, 309.11, 277.4100000000001, 689.9]",191385.1590,"[1125.14, 361.38, 1, 1135.3, 345.97, 1, 1109.3...",7.503957,7.600723,6.263417,0.875935,0.673371,...,0.879295,0.803033,0.740115,0.884123,0.938004,0.893016,0.865615,"1132.1380615234375,357.40948486328125,2,1138.9...","1118.97,377.231,2,1133.62,362.538,2,1115.89,36...","1122.510498046875,360.0570373535156,2,1136.302..."


In [15]:
import argparse

from KeyFrameDetector.key_frame_detector import keyframeDetection


keyframeDetection('../data/videos/Athletics_Mixed_Tokyo_2020_20_1.mp4', '../data/videos', float(0.3))
